# An introduction to Predictive Modeling in Python

This notebook walks through an example of using supervised learning on structured data.

Start by importing some basic libraries. ```%matplotlib inline``` will allow us to print some charts inside the notebook later using ```matplotlib.pyplot``` later. ```numpy``` is a library to manage large arrays. ```pandas``` is a library for data manipulation and analytics. These are the foundation for doing data preparation for AI/ML.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Loading tabular data from the Titanic kaggle challenge in a pandas Data Frame

Let us have a look at the Titanic dataset from the Kaggle Getting Started challenge at:

https://www.kaggle.com/c/titanic-gettingStarted

We can load the CSV file as a pandas data frame in one line. The code below uses `os.path.abspath()` to find the location of this Notebook as a way or finding where the dataset is stored. This may not be portable, so you may have to update the path below with the location of your data.

In [ ]:
#data = pd.read_csv('../datasets/titanic_train.csv')
import os
os.path.abspath("")

data = pd.read_csv(os.path.abspath("") + '/data/train.csv')

```pandas``` data frames have a HTML table representation in Jupyter Notebook.s Let's have a look at the first 6 rows:

In [ ]:
data.head(6)

In [ ]:
data.count()

The data frame has 891 rows. Some passengers have missing information though: in particular Age and Cabin info can be missing. The meaning of each the columns is explained on the challenge website:

https://www.kaggle.com/c/titanic-gettingStarted/data

A data frame can be converted into a numpy array by calling the `values` attribute:

In [ ]:
data.values

However this cannot be directly fed to a scikit-learn model:


- the target variable (survival) is mixed with the input data

- some attribute such as unique ids have no predictive values for the task

- the values are heterogeneous (string labels for categories, integers and floating point numbers)

- some attribute values are missing (NaN: "not a number")

### Questions:

* Which fields do you think will be most predictive? (Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked)

* How would you handle missing values in the data?

## Predicting survival

The goal of the Kaggle challenge is to predict whether a passenger has survived from others known attribute. Let us have a look at the `Survived` columns:

In [ ]:
data.Survived.dtype

In [ ]:
np.mean(data.Survived == 0)

From this the subset of the full passengers list, about 2/3 perished in the event. So if we are to build a predictive model from this data, a baseline model to compare the performance to would be to always predict death. Such a constant model would reach around 62% predictive accuracy (which is higher than predicting at random):

pandas `Series` (columns) instances can be converted to regular 1D numpy arrays by using the `values` attribute:

In [ ]:
target = data.Survived.values
type(target)

In [ ]:
target.dtype

The Survived field is technically a Boolean. We could have told pandas that when we imported the data by passing a Schema, but we can cast the array now as well:

In [ ]:
data['Survived'] = data['Survived'].astype('bool')
target_bool = target.astype(dtype=bool)
print("numpy array datatype:", target_bool.dtype)
print("numpy array values:",target[:6])
print("pandas DataFrame field values:\n",data['Survived'].head(6))

## Training a predictive model on numerical features

`sklearn` works with homegeneous numerical feature descriptors passed as a numpy array. Therefore passing the raw data frame will not work out of the box.

Let us start simple and build a first model that only uses readily available numerical features as input, namely `data.Fare`, `data.Pclass` and `data.Age`.

In [ ]:
numerical_features = data.get(['Fare', 'Pclass', 'Age'])
numerical_features.head(6)

Unfortunately some passengers do not have age information:

In [ ]:
numerical_features.count()

Let's use pandas to get the median value for the data we do have and then the `fillna` method to input the median age for those passengers:

In [ ]:
median_features = numerical_features.dropna().median()
median_features

In [ ]:
imputed_features = numerical_features.fillna(median_features)
imputed_features.count()

In [ ]:
imputed_features.head(6)

Now that the data frame is clean, we can convert it into an homogeneous numpy array of floating point values:

In [ ]:
features_array = imputed_features.values
features_array

Let's take the 80% of the data for training a first model and keep 20% for computing is generalization score:

In [ ]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(
    features_array, target_bool, test_size=0.20, random_state=20)

In [ ]:
print("feature_train:".ljust(30), features_train.shape)
print("feature_test:".ljust(30), features_test.shape)
print("target_train:".ljust(30), target_train.shape)
print("target_test:".ljust(30), target_test.shape)

Let's start with a simple model from sklearn, `LogisticRegression`:

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_initial = LogisticRegression()
logreg_initial.fit(features_train, target_train)

The ```fit``` function trains the model using the features in our training set(```features_train```), and the expected prediction for each passenger (```target_train```). Now let's perform inference using using the records in our test set (```features_test```):

In [ ]:
target_predicted = logreg_initial.predict(features_test)

In a lot of ML examples you'll see `targeted_predicted` named `y_hat`, which in statistics is written ŷ (technically a 'y' with a circumflex). This is the name for the "predicted value of Y" for regression equitions.

Now we can compare our model's predictions with the actual data:

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(target_test, target_predicted)

### Questions:

* How does this compare with our baseline prediction of 61.6%?

## Saving and Loading Models

In the previous section we trained('fit') a model and performed inference('predict') back to back. In many cases training takes a significant amount of time and then the model is saved and re-loaded for use later - potentially on different system(s).

Let's save our LinearRegression model:

In [ ]:
import pickle

# save model for use later
with open('model.pkl','wb') as f:
    pickle.dump(logreg_initial,f)

Check and see that the model was written out to a file:

In [ ]:
%%sh
ls -lh model.pkl

Now let's read the model back in and use it to perform the prediction again:

In [ ]:
# read model back in
with open('model.pkl', 'rb') as f:
    logreg_reloaded = pickle.load(f)

target_repredicted = logreg_reloaded.predict(features_test)
accuracy_score(target_test, target_repredicted)

Let's try a different mix of the data set by re-splitting our dataset with a different ```random_state```.

In [ ]:
new_features_train, new_features_test, new_target_train, new_target_test = train_test_split(
    features_array, target_bool, test_size=0.20, random_state=10)

new_target_predicted = logreg_reloaded.predict(new_features_test)
accuracy_score(new_target_test, new_target_predicted)

### Questions:

* What are some reasons why the accuracy is different between the two sets of feature_test data?

## Model evaluation and interpretation

### Interpreting linear model weights

The `coef_` attribute of a fitted linear model such as `LogisticRegression` holds the weights of each features:

In [ ]:
print("coef:", logreg_initial.coef_)

x = np.arange(len(numerical_features.columns.values))
plt.bar(numerical_features.columns.values, logreg_initial.coef_.ravel())
_ = plt.xticks(numerical_features.columns.values, rotation=30)

### Questions:

* What does the importance of these features mean for the passengers of the Titanic?

### Alternative evaluation metrics

Logistic Regression is a probabilistic models: instead of just predicting a binary outcome (survived or not) given the input features it can also estimates the posterior probability of the outcome given the input features using the `predict_proba` method:

In [ ]:
target_predicted_proba = logreg_initial.predict_proba(features_test)
target_predicted_proba[:6]

By default the decision threshold is 0.5: if we vary the decision threshold from 0 to 1 we could generate a family of binary classifier models that address all the possible trade offs between false positive and false negative prediction errors.

We can summarize the performance of a binary classifier for all the possible thresholds by plotting the ROC curve and quantifying the Area under the ROC curve:

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = auc(fpr, tpr)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")

In [ ]:
plot_roc_curve(target_test, target_predicted_proba)

Here the area under ROC curve is 0.722 which is very similar to the accuracy (0.732). The ROC-AUC score of a random model is expected to be 0.5 on average while the accuracy score of a random model depends on the class imbalance of the data. ROC-AUC can be seen as a way to callibrate the predictive accuracy of a model against class imbalance.

It is possible to see the details of the false positive and false negative errors by computing the confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(target_test, target_predicted, labels=logreg_initial.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=logreg_initial.classes_)
disp.plot()

Another way to quantify the quality of a binary classifier on imbalanced data is to compute the precision, recall and f1-score of a model (at the default fixed decision threshold of 0.5).

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(target_test, target_predicted,
                            target_names=['not survived', 'survived']))

Precision is the measure of the number of True Positives relative to the total number of Positive outcomes predicted: TP / (TP + FP).

Recall is the measure of the number of True Positives relative to the number of ground truth Positives: TP / (TP + FN)

F1-Score balances the Precision and Recall scores: F1 = 2 * (Precision * Recall) / (Precision + Recall)

Support gives the number of how many of each kind of record were in our dataset.

### Cross-validation

We previously decided to randomly split the data to evaluate the model on 20% of held-out data. However the location randomness of the split might have a significant impact on the estimated accuracy:

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features_array, target_bool, test_size=0.20, random_state=0)

logreg_initial.fit(features_train, target_train).score(features_test, target_test)

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=1)

logreg_initial.fit(features_train, target_train).score(features_test, target_test)

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=2)

logreg_initial.fit(features_train, target_train).score(features_test, target_test)

So instead of using a single train / test split, we can use a group of them and compute the min, max and mean scores as an estimation of the real test score while not underestimating the variability:

In [ ]:
%%time

from sklearn.model_selection import cross_val_score

scores = cross_val_score(logreg_initial, features_array, target_bool, cv=3)
scores

In [ ]:
scores.min(), scores.mean(), scores.max()

`cross_val_score` reports accuracy by default but it can also be used to report other performance metrics such as ROC-AUC or f1-score:

In [ ]:
%%time

scores = cross_val_score(logreg_initial, features_array, target_bool, cv=3,
                         scoring='roc_auc')
scores.min(), scores.mean(), scores.max()

**Exercise**:

- Compute cross-validated scores for other classification metrics ('precision', 'recall', 'f1', 'accuracy'...).

- Change the number of cross-validation folds between 3 and 10: what is the impact on the mean score? on the processing time?

Hints:

The list of classification metrics is available in the online documentation:

  http://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values

## More feature engineering and richer models

Let us now try to build richer models by including more features as potential predictors for our model.

Categorical variables such as `data.Embarked` or `data.Sex` can be converted as boolean indicators features also known as dummy variables or one-hot-encoded features:

In [ ]:
pd.get_dummies(data.Sex, prefix='Sex').head(5)

In [ ]:
pd.get_dummies(data.Embarked, prefix='Embarked').head(5)

We can combine those new numerical features with the previous features using `pandas.concat` along `axis=1`:

In [ ]:
rich_features = pd.concat([data.get(['Fare', 'Age','Pclass']),
                           pd.get_dummies(data.Sex, prefix='Sex'),
                           pd.get_dummies(data.Embarked, prefix='Embarked')],
                          axis=1)
rich_features.head(6)

In this case, by construction the new `Sex_male` feature is redundant with `Sex_female`. Let's drop it:

In [ ]:
rich_features_no_male = rich_features.drop(columns='Sex_male', axis=1)
rich_features_no_male.head(6)

Let us not forget to imput the median age for passengers without age information:

In [ ]:
rich_features_final = rich_features_no_male.fillna(rich_features_no_male.dropna().median())
rich_features_final.head(6)

We can finally cross-validate a logistic regression model on this new data and observe that the mean score has significantly increased:

In [ ]:
%%time

logreg = LogisticRegression(max_iter=225)
scores = cross_val_score(logreg, rich_features_final, target_bool, cv=5, scoring='accuracy')
print(scores.min(), scores.mean(), scores.max())

Let's plot the weights for the features of this newly fitted logistic regression model:

In [ ]:
logreg.fit(rich_features_final, target_bool)

print("coef:", logreg.coef_)

x = np.arange(len(numerical_features.columns.values))
plt.bar(rich_features_final.columns.values, logreg.coef_.ravel())
_ = plt.xticks(rich_features_final.columns.values, rotation=30)

### Questions:

* How do you interpret the importance of the new features?

* Do you think this trained model would be accurate with newer data? e.g. passenger data from a ship that sank today.

### Training Non-linear models: ensembles of randomized trees

`sklearn` also implements non linear models that are known to perform very well for data-science projects where datasets don't have a huge number of features (e.g. less than 5000). A full list of `sklearn` supervised learning models can be found here: https://scikit-learn.org/stable/supervised_learning.html

Let us have a look at Random Forests and Gradient Boosted Trees:

In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rf, rich_features_final, target_bool, cv=5, n_jobs=4,
                         scoring='accuracy')
print(scores.min(), scores.mean(), scores.max())

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                subsample=.8, max_features=.5)
gb_scores = cross_val_score(gb, rich_features_final, target_bool, cv=5, n_jobs=4,
                         scoring='accuracy')
print(gb_scores.min(), gb_scores.mean(), gb_scores.max())

Both models seem to do slightly better than the logistic regression model on this data.

**Exercise**:

- Change the value of the learning_rate and other `GradientBoostingClassifier` parameter, can you get a better mean score?

- Would treating the `PClass` variable as categorical improve the models performance?

- Find out which predictor variables (features) are the most informative for those models.

Hints:

Fitted ensembles of trees have `feature_importance_` attribute that can be used similarly to the `coef_` attribute of linear models.

In [ ]:
rf.fit(rich_features_final, target)
rf.feature_importances_
rf_feature_importances = pd.DataFrame(rf.feature_importances_, index=rich_features_final.columns,  columns=['importance']).sort_values('importance', ascending=False)

x = np.arange(len(rich_features_final.columns))
plt.bar(x, rf.feature_importances_.ravel())
_ = plt.xticks(x, rich_features_final.columns, rotation=30)

In [ ]:
gb.fit(rich_features_final, target)
gb.feature_importances_
gb_feature_importances = pd.DataFrame(gb.feature_importances_, index=rich_features_final.columns,  columns=['importance']).sort_values('importance', ascending=False)

x = np.arange(len(rich_features_final.columns))
plt.bar(x, gb.feature_importances_.ravel())
_ = plt.xticks(x, rich_features_final.columns, rotation=30)

## Automated parameter tuning

Instead of changing the value of the learning rate manually and re-running the cross-validation, we can find the best values for the parameters automatically (assuming we are willing to wait):

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV

gb = GradientBoostingClassifier(n_estimators=100, subsample=.8)

params = {
    'learning_rate': [0.05, 0.1, 0.5],
    'max_features': [0.5, 1],
    'max_depth': [3, 4, 5],
}
gs = GridSearchCV(gb, params, cv=5, scoring='roc_auc', n_jobs=4)
gs.fit(rich_features_final, target)

Let us sort the models by mean validation score:

In [ ]:
gs.cv_results_['mean_test_score']

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

We should note that the mean scores are very close to one another and almost always within one standard deviation of one another. This means that all those parameters are quite reasonable.

## Avoiding data snooping with pipelines

When doing imputation in pandas, prior to computing the train test split we use data from the test to improve the accuracy of the median value that we impute on the training set. This is actually cheating. To avoid this we should compute the median of the features on the training fold and use that median value to do the imputation both on the training and validation fold for a given CV split.

To do this we can prepare the features as previously but without the imputation: we just replace missing values by the -1 marker value:

In [ ]:
features = pd.concat([data.get(['Fare', 'Age', 'Pclass']),
                      pd.get_dummies(data.Sex, prefix='Sex'),
                      pd.get_dummies(data.Embarked, prefix='Embarked')],
                     axis=1)
features = features.drop(columns='Sex_male', axis=1)

# Because of the following bug we cannot use NaN as the missing
# value marker, use a negative value as marker instead:
# https://github.com/scikit-learn/scikit-learn/issues/3044
features = features.fillna(-1)
features.head(6)

We can now use the `Imputer` transformer of scikit-learn to find the median value on the training set and apply it on missing values of both the training set and the test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(features.values, random_state=1)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median', missing_values=-1)

imputer.fit(X_train)

The median age computed on the training set is stored in the `statistics_` attribute.

In [ ]:
print("Median age of full set:".ljust(30), median_features['Age'])
print("Median age of training set:".ljust(30), imputer.statistics_[1])

Imputation can now happen by calling  the transform method:

In [ ]:
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [ ]:
np.any(X_train == -1)

In [ ]:
np.any(X_train_imputed == -1)

In [ ]:
np.any(X_test == -1)

In [ ]:
np.any(X_test_imputed == -1)

We can now use a pipeline that wraps an imputer transformer and the classifier itself:

In [ ]:
from sklearn.pipeline import Pipeline

imputer = SimpleImputer(strategy='median', missing_values=-1)

classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                        subsample=.8, max_features=.5)

pipeline = Pipeline([
    ('imp', imputer),
    ('clf', classifier),
])

scores = cross_val_score(pipeline, features.values, target, cv=5, n_jobs=4,
                         scoring='accuracy', )
print("Scores with Snooping:".ljust(25), gb_scores.min(), gb_scores.mean(), gb_scores.max())
print("Scores without Snooping:".ljust(25), scores.min(), scores.mean(), scores.max())

The mean cross-validation is slightly lower than when we used the imputation on the whole data as we did earlier although not by much. This means that in this case the data-snooping was not really helping the model cheat by much.

Let us re-run the grid search, this time on the pipeline. Note that thanks to the pipeline structure we can optimize the interaction of the imputation method with the parameters of the downstream classifier without cheating:

In [ ]:
%%time

params = {
    'imp__strategy': ['mean', 'median'],
    'clf__max_features': [0.5, 1],
    'clf__max_depth': [3, 4, 5],
}
gs = GridSearchCV(pipeline, params, cv=5, scoring='roc_auc', n_jobs=4)
gs.fit(features, target)

In [ ]:
gs.cv_results_['mean_test_score']

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

From this search we can conclude the best imputation strategy and tuning parameters when training a GBRT model on this data given the parameter search space we defined.

## Final Thoughts

The AI Ecosystem is evolving quickly! There is a huge number of tools available some gaining favor, some losing favor. Even individual tools are being updated constantly and documentation and examples you find that are even 1 year old may be out of date and no longer work correctly with current versions of a toolchain you install.

Also, don't expect one model to do everything! Break down the problem you're trying to solve and have different models for different steps. This is a common practice and is called building an 'Ensemble.' In the

The most important thing is to understand the fundementals, know what to look for to make sure your model is performing well and to avoid common pitfalls -- and, finally, don't be afraid to try something different. Good luck!

## Credits

Many thanks to:

- Adam Walz for providing the original version of this notebook https://github.com/adamwalz/Jupyter-Notebooks

Adam thanks:

- Kaggle for setting up the Titanic challenge.

- This blog post by Philippe Adjiman for inspiration:

http://www.philippeadjiman.com/blog/2013/09/12/a-data-science-exploration-from-the-titanic-in-r/

## Additional Resources

- Andrew Ng
  - https://www.coursera.org/learn/ai-for-everyone
  - https://deeplearning.ai


- Data Science Challenge Websites / Places To Find Example Datasets
  - https://kaggle.com
  - https://www.drivendata.org
  - https://competitions.codalab.org/competitions/
  - https://machinehack.com/hackathons
  - https://tianchi.aliyun.com/competition/gameList/activeList


- AutoML
  - https://neptune.ai/blog/a-quickstart-guide-to-auto-sklearn-automl-for-machine-learning-practitioners


- Examples of Specific Kinds of Data Science Challenges
  - Image Segmentation
    - [Open Images Instance Segmentation RVC 2020 edition | Kaggle](https://www.kaggle.com/competitions/open-images-instance-segmentation-rvc-2020)
    - [DanceTrack : Tracking Multiple Objects in Uniform Appearance & Diverse Motion | CodaLab](https://competitions.codalab.org/competitions/35786)
  - Image Classification
    - [Conser-vision Practice Area: Image Classification | DrivenData](https://www.drivendata.org/competitions/87/competition-image-classification-wildlife-conservation/)
    - [Petals to the Metal - Flower Classification on TPU | Kaggle](https://www.kaggle.com/competitions/tpu-getting-started/overview/description)
  - Natural Language Processing
    - [Natural Language Processing with Disaster Tweets | Kaggle](https://www.kaggle.com/competitions/nlp-getting-started/overview/description)
    - [Build a complex Named Entity Recognition System for 11 languages | CodaLab](https://competitions.codalab.org/competitions/36425)
    - [Sentiment Analysis: Weekend Hackathon Edition #2 — The Last Hacker Standing | Machine Hack](https://machinehack.com/hackathons/sentiment_analysis_weekend_hackathon_edition_2_the_last_hacker_standing/overview)
  - Prediction
    - [Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines | Driven Data](https://www.drivendata.org/competitions/66/flu-shot-learning/)
  - Generative Adversarial Networks
    - [I’m Something of a Painter Myself | Kaggle](https://www.kaggle.com/competitions/gan-getting-started)